<a href="https://colab.research.google.com/github/alouvre/CapstoneNeeds/blob/main/Data_Collecting_FiturPrediksiPolaPengeluaranUser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set Kaggle

# menginstall kaggle
!pip install -q kaggle
# membuat direktori
!mkdir ~/.kaggle
# menyalin kaggle.json ke dir kaggle
!cp kaggle.json ~/.kaggle/
# meminta izin untuk akses
!chmod 600 ~/.kaggle/kaggle.json

!kaggle -h

In [ ]:
# import zipfile

# # Mengunduh dataset
# !kaggle datasets download -d 'shivamb/bank-customer-segmentation'

# # Membuat direktori dataset
# # !mkdir dataset

# # Mengekstraksi file zip
# nama_file_zip = 'bank-customer-segmentation.zip'
# nama_folder_tujuan = 'dataset'
# dataset_zip = zipfile.ZipFile(nama_file_zip,'r')
# dataset_zip.extractall('dataset')
# dataset_zip.close()

# # Memindahkan files ke direktori dataset
# !mv bank-customer-segmentation.zip dataset

In [ ]:
import pandas as pd

# Load dataset from Project folder
df = pd.read_csv('dataset/bank_transactions.csv')

# To stretch head function output to the notebook width
pd.set_option('display.max_columns', None)

df.head()

In [ ]:
df.shape

In [ ]:
# df['Credit_Score'].unique()

In [ ]:
# df[[''],['Credit_Score']]

In [ ]:
data = df[['CustomerID','CustAccountBalance', 'TransactionAmount (INR)']]
data.head()

In [ ]:
data = data.rename(columns={'CustomerID': 'IDUser', 'CustAccountBalance': 'SaldoUser', 'TransactionAmount (INR)': 'JumlahTransaksi'})
data.head()

In [ ]:
data.info()

In [ ]:
# Definisikan kurs pertukaran (contoh: 1 INR = 188,42 IDR)
inr_to_idr_exchange_rate = 188.42

# Buat kolom baru untuk jumlah transaksi dalam IDR
data['JumlahTransaksi'] = data['JumlahTransaksi'].astype(float)
data['JumlahTransaksi'] = data['JumlahTransaksi'] * inr_to_idr_exchange_rate

# Buat kolom baru untuk saldo pengguna dalam IDR
data['SaldoUser'] = data['SaldoUser'].astype(float)
data['SaldoUser'] = data['SaldoUser'] * inr_to_idr_exchange_rate

In [ ]:
data.head()

In [ ]:
pd.options.display.float_format = '{:.0f}'.format
# pd.options.display.float_format = None

# Mengonversi nilai saldo pengguna dari format ilmiah ke float
data['SaldoUser'] = data['SaldoUser'].apply(lambda x: float(f"{x:.10f}"))

In [ ]:
data.head()